In [1]:
import numpy as np
import pandas as pd
import numbers
from copy import *
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# TODO
pd.set_option('display.max_columns', None)
#data = pd.read_csv("clinvar_conflicting.csv")
data = pd.read_table("clinvar_conflicting_1.txt",sep=',')
data1 = deepcopy(data)


data1

C:\Users\Ajangu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,CLNDNINCL,CLNHGVS,CLNSIGINCL,CLNVC,CLNVI,MC,ORIGIN,SSR,CLASS,Allele,Consequence,IMPACT,SYMBOL,Feature_type,Feature,BIOTYPE,EXON,INTRON,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,DISTANCE,STRAND,BAM_EDIT,SIFT,PolyPhen,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,1,1168180,G,C,0.0771,0.10020,0.1066,MedGen:CN169374,NaN,not_specified,NaN,NC_000001.10:g.1168180G>C,NaN,single_nucleotide_variant,UniProtKB_(protein):Q96L58#VAR_059317,SO:0001583|missense_variant,1,NaN,0,C,missense_variant,MODERATE,B3GALT6,Transcript,NM_080605.3,protein_coding,1/1,NaN,552,522,174,E/D,gaG/gaC,NaN,1.0,NaN,tolerated,benign,NaN,NaN,NaN,NaN,NaN,1.053,-0.208682,2.0
1,1,1470752,G,A,0.0000,0.00000,0.0000,"MedGen:C1843891,OMIM:607454,Orphanet:ORPHA9877...",NaN,Spinocerebellar_ataxia_21|not_provided,NaN,NC_000001.10:g.1470752G>A,NaN,single_nucleotide_variant,OMIM_Allelic_Variant:616101.0001|UniProtKB_(pr...,SO:0001583|missense_variant,1,NaN,0,A,missense_variant,MODERATE,TMEM240,Transcript,NM_001114748.1,protein_coding,4/4,NaN,523,509,170,P/L,cCg/cTg,NaN,-1.0,OK,deleterious_low_confidence,benign,NaN,NaN,NaN,NaN,NaN,31.000,6.517838,-3.0
2,1,1737942,A,G,0.0000,0.00001,0.0000,"Human_Phenotype_Ontology:HP:0000486,MedGen:C00...",NaN,Strabismus|Nystagmus|Hypothyroidism|Intellectu...,NaN,NC_000001.10:g.1737942A>G,NaN,single_nucleotide_variant,OMIM_Allelic_Variant:139380.0002|UniProtKB_(pr...,"SO:0001583|missense_variant,SO:0001623|5_prime...",35,NaN,1,G,missense_variant,MODERATE,GNB1,Transcript,NM_002074.4,protein_coding,6/12,NaN,632,239,80,I/T,aTc/aCc,NaN,-1.0,OK,deleterious,probably_damaging,NaN,NaN,NaN,NaN,NaN,28.100,6.061752,-1.0
3,1,2160305,G,A,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002...",NaN,Shprintzen-Goldberg_syndrome|not_provided,NaN,NC_000001.10:g.2160305G>A,NaN,single_nucleotide_variant,OMIM_Allelic_Variant:164780.0004|UniProtKB_(pr...,SO:0001583|missense_variant,33,NaN,0,A,missense_variant,MODERATE,SKI,Transcript,XM_005244775.1,protein_coding,1/7,NaN,132,100,34,G/S,Ggc/Agc,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.500,3.114491,NaN
4,1,2160305,G,T,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002",NaN,Shprintzen-Goldberg_syndrome,NaN,NC_000001.10:g.2160305G>T,NaN,single_nucleotide_variant,OMIM_Allelic_Variant:164780.0005|UniProtKB_(pr...,SO:0001583|missense_variant,33,NaN,0,T,missense_variant,MODERATE,SKI,Transcript,XM_005244775.1,protein_coding,1/7,NaN,132,100,34,G/C,Ggc/Tgc,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.700,4.766224,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65183,X,154158201,T,G,0.0801,0.13923,0.1605,"MedGen:C0019069,OMIM:306700,SNOMED_CT:28293008...",NaN,Hereditary_factor_VIII_deficiency_disease|not_...,NaN,NC_000023.10:g.154158201T>G,NaN,single_nucleotide_variant,"ARUP_Laboratories,_Molecular_Genetics_and_Geno...",SO:0001819|synonymous_variant,1,NaN,0,G,synonymous_variant,LOW,F8,Transcript,NM_000132.3,protein_coding,14/26,NaN,4035,3864,1288,S,tcA/tcC,NaN,-1.0,OK,NaN,NaN,NaN,NaN,NaN,NaN,0.00158,0.105,-0.630908,NaN
65184,X,154159118,C,T,0.0020,0.00060,0.0013,MedGen:CN169374|MedGen:CN239152,NaN,"not_specified|Hemophilia_A,_FVIII_Deficiency",NaN,NC_000023.10:g.154159118C>T,NaN,single_nucleotide_variant,"ARUP_Laboratories,_Molecular_Genetics_and_Geno...",SO:0001583|missense_variant,1,NaN,1,T,missense_variant,MODERATE,F8,Transcript,NM_000132.3,protein_coding,14/26,NaN,3118,2947,983,V/I,Gta/Ata,NaN,-1.0,OK,tolerated,benign,NaN,NaN,NaN,NaN,0.00158,0.002,-1.731470,3.0
65185,X,154194886,C,T,0.0125,0.00370,0.0111,MedGen:CN169374|MedGen:CN239152,NaN,"not_specified|Hemophilia_A,_FVIII_Deficiency",NaN,NC_000023.10:g.154194886C>T,NaN,single_nucleotide_variant,"ARUP_Laboratories,_Molecular_Genetics_and_Geno...",SO:0001819|synony

In [2]:
#ISIKLIKKU LÄHENEMIST VAJAVAD TULBAD

#muudame nende tulpade sisu
data1[["Amino_acids_initial", "Amino_acids_replacement"]] = data1.Amino_acids.str.split("/",expand=True)
data1[["Codons_initial", "Codons_replacement"]] = data1.Codons.str.split("/",expand=True)

data1["EXON"]=data1["EXON"].astype(str)
data1["INTRON"]=data1["INTRON"].astype(str)

#data1["EXON"].isnull().sum()
data1.loc[data1["EXON"] == data1["INTRON"], ["INTRON", "EXON"]] = np.nan
data1.loc[(data1["EXON"]!='nan') & (data1["INTRON"]!='nan') & (data1["REF"].str.len() == 1), ["INTRON", "EXON"]] = np.nan

#viskame välja kõik nan valued
data1 = data1[data1['EXON'].notna()]
data1 = data1[data1['INTRON'].notna()]

data1[["EXON_start", "EXON_end"]] = data1.EXON.str.split("/",expand=True)
data1[["INTRON_start", "INTRON_end"]] = data1.INTRON.str.split("/",expand=True)


##CHROM
data1["CHROM"]=data1["CHROM"].astype(str)
data1.loc[data1["CHROM"] == "X", "CHROM"] = "23"
data1.loc[data1["CHROM"] == "MT", "CHROM"] = "24"

In [3]:
#LIHTSASTI DEFINEERITAVAD TULBAD
inttüüpi = ["POS", "ORIGIN", "CLASS", "CHROM"]
floattüüpi = ["AF_ESP", "AF_EXAC", "AF_TGP", "LoFtool", "CADD_PHRED", "CADD_RAW" ,  "STRAND", "BLOSUM62", "EXON_start", "EXON_end", "INTRON_start", "INTRON_end"]
strtüüpi = ["REF", "ALT", "Consequence", "IMPACT", "SYMBOL", "BIOTYPE" , "CLNVC", "IMPACT", "SIFT", "PolyPhen", "Amino_acids_initial", "Amino_acids_replacement", "Codons_initial", "Codons_replacement", "cDNA_position" , "CDS_position", "Protein_position"]


def datatypes(inttüüpi, strtüüpi, floattüüpi):
  for i in range(len(inttüüpi)):
      data1[inttüüpi[i]]=data1[inttüüpi[i]].astype(int)

  for i in range(len(strtüüpi)):
      data1[strtüüpi[i]]=data1[strtüüpi[i]].astype(str)
  
  for i in range(len(floattüüpi)):
      data1[floattüüpi[i]]=data1[floattüüpi[i]].astype(float).round(decimals=5)
      
datatypes(inttüüpi, strtüüpi, floattüüpi)

In [4]:
#CDNA, CDS ja protein tulpade töötlus
data1[["cDNA_pos_start" , "cDNA_pos_end"]] = data1.cDNA_position.str.split("-",expand=True)
data1[["CDS_pos_start" , "CDS_pos_end"]] = data1.CDS_position.str.split("-",expand=True)
data1[["Protein_pos_start" , "Protein_pos_end"]] = data1.Protein_position.str.split("-",expand=True)

data1.loc[data1["cDNA_pos_end"].isnull(), "cDNA_pos_end"] = data1["cDNA_pos_start"]
data1.loc[data1["CDS_pos_end"].isnull(), "CDS_pos_end"] = data1["CDS_pos_start"]
data1.loc[data1["Protein_pos_end"].isnull(), "Protein_pos_end"] = data1["Protein_pos_start"]

data1.loc[data1["cDNA_pos_end"].str.contains("?", regex=False, na=False), "cDNA_pos_end"] = np.nan
data1.loc[data1["CDS_pos_end"].str.contains("?", regex=False, na=False), "CDS_pos_end"] = np.nan
data1.loc[data1["Protein_pos_end"].str.contains("?", regex=False, na=False), "Protein_pos_end"] = np.nan

data1.loc[data1["cDNA_pos_start"].str.contains("?", regex=False, na=False), "cDNA_pos_start"] = np.nan
data1.loc[data1["CDS_pos_start"].str.contains("?", regex=False, na=False), "CDS_pos_start"] = np.nan
data1.loc[data1["Protein_pos_start"].str.contains("?", regex=False, na=False), "Protein_pos_start"] = np.nan

floatiks_ja_nan_välja = ["cDNA_pos_start" , "cDNA_pos_end", "CDS_pos_start" , "CDS_pos_end", "Protein_pos_start" , "Protein_pos_end"]
for i in range(len(floatiks_ja_nan_välja)):
  data1[floatiks_ja_nan_välja[i]]=data1[floatiks_ja_nan_välja[i]].astype(float).round(decimals=5)

for i in range(len(floatiks_ja_nan_välja)):
  data1 = data1[data1[floatiks_ja_nan_välja[i]].notna()]

In [5]:
välja = ["cDNA_position","CDS_position","Protein_position","Allele", "Amino_acids", "Codons", "EXON","INTRON", "CLNDN", "CLNDISDB","CLNDNINCL", "CLNSIGINCL", "CLNDISDBINCL", "CLNHGVS", "CLNVI", "MC", "SSR", "Feature_type", "Feature", "DISTANCE", "BAM_EDIT", "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE"]
for i in välja:
    data1 = data1.drop(labels=[i], axis=1)

In [6]:
#asendame nendes tulpades nan väärtused tulba keskmisega
keskmised=["LoFtool","CADD_PHRED","CADD_RAW","BLOSUM62"]
for i in range(len(keskmised)):
    data1.loc[data1[keskmised[i]].notna()==False, keskmised[i]] = data1[keskmised[i]].mean()

#asendame nendes tulpades nan väärtused 0-ga
for i in range(len(floattüüpi)):
    data1.loc[data1[floattüüpi[i]].notna()==False, floattüüpi[i]] = 0
        
data1.AF_ESP[data1.AF_ESP == 0].count()

30622

In [7]:
mitteInt = floattüüpi + strtüüpi

valja = ["cDNA_position" , "CDS_position", "Protein_position"]
for i in valja:
    mitteInt.remove(i)

for i in range(len(mitteInt)):
    if data1[mitteInt[i]].isnull().values.any():
        print(mitteInt[i])

['CHROM', 'POS', 'REF', 'ALT', 'AF_ESP', 'AF_EXAC', 'AF_TGP', 'CLNVC', 'ORIGIN', 'CLASS', 'Consequence', 'IMPACT', 'SYMBOL', 'BIOTYPE', 'STRAND', 'SIFT', 'PolyPhen', 'LoFtool', 'CADD_PHRED', 'CADD_RAW', 'BLOSUM62', 'Amino_acids_initial', 'Amino_acids_replacement', 'Codons_initial', 'Codons_replacement', 'EXON_start', 'EXON_end', 'INTRON_start', 'INTRON_end', 'cDNA_pos_start', 'cDNA_pos_end', 'CDS_pos_start', 'CDS_pos_end', 'Protein_pos_start', 'Protein_pos_end']


CHROM                      0
POS                        0
REF                        0
ALT                        0
AF_ESP                     0
AF_EXAC                    0
AF_TGP                     0
CLNVC                      0
ORIGIN                     0
CLASS                      0
Consequence                0
IMPACT                     0
SYMBOL                     0
BIOTYPE                    0
STRAND                     0
SIFT                       0
PolyPhen                   0
LoFtool                    0
CADD_PHRED                 0
CADD_RAW                   0
BLOSUM62                   0
Amino_acids_initial        0
Amino_acids_replacement    0
Codons_initial             0
Codons_replacement         0
EXON_start                 0
EXON_end                   0
INTRON_start               0
INTRON_end                 0
cDNA_pos_start             0
cDNA_pos_end               0
CDS_pos_start              0
CDS_pos_end                0
Protein_pos_start          0
Protein_pos_en

In [8]:
#onehotencoding
data1 = pd.get_dummies(data1,columns=["REF", "ALT", "CLNVC", "Consequence", "IMPACT", "SYMBOL", "BIOTYPE","SIFT",
                                      "PolyPhen","Amino_acids_initial", 
                                      "Amino_acids_replacement","Codons_initial","Codons_replacement"], dummy_na=True)

In [9]:
tulbad = list(data1.columns)
for i in range(len(tulbad)):
    if data1[tulbad[i]].isnull().values.any():
        print(tulbad[i])
        
data1.isnull().sum()

CHROM                      0
POS                        0
AF_ESP                     0
AF_EXAC                    0
AF_TGP                     0
                          ..
Codons_replacement_ttg     0
Codons_replacement_ttgg    0
Codons_replacement_ttgt    0
Codons_replacement_ttt     0
Codons_replacement_nan     0
Length: 6410, dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data1.drop(labels='CLASS',axis=1),data1.CLASS,train_size=0.7,random_state=0)


#mudeli fittimine
rf = RandomForestClassifier(n_estimators=100,max_depth=4,max_features="log2",random_state=0).fit(X_train, y_train)

In [11]:
y_pred = rf.predict(X_test)
acc_score=accuracy_score(y_pred, y_test)
print("Accuracy score:\n%s " %round(acc_score*100,2))

Accuracy score:
75.09 
